# HQDE System Test Notebook

This notebook tests the fixed HQDE (Hierarchical Quantum-Distributed Ensemble Learning) system to verify it works dynamically instead of showing static output.

## What this tests:
1. Dynamic training vs static simulation
2. Real model predictions instead of random outputs
3. Adaptive quantization functionality
4. Quantum-inspired aggregation
5. Synthetic data generation

## Run all cells in order to test the system

## 1️⃣ Setup and Imports

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add the project directory to Python path
# Update this path if your notebook is in a different location
project_dir = './'  # Change if needed
sys.path.insert(0, project_dir)

print("Setting up HQDE Test Environment...")
print(f"Project directory: {os.path.abspath(project_dir)}")

# Test basic imports
try:
    import torch
    import torch.nn as nn
    import numpy as np
    print("SUCCESS: PyTorch and NumPy imported successfully")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError as e:
    print(f"ERROR: Import failed: {e}")
    print("Please install: pip install torch numpy")

## 2️⃣ Test HQDE Core Components

In [ ]:
# Test HQDE core components
try:
    # Add core directory to path
    sys.path.insert(0, os.path.join(project_dir, 'hqde', 'core'))
    from hqde_system import AdaptiveQuantizer, QuantumInspiredAggregator
    print("SUCCESS: HQDE Core components imported successfully")
    
    # Test AdaptiveQuantizer
    print("\nTesting AdaptiveQuantizer...")
    quantizer = AdaptiveQuantizer(base_bits=8, min_bits=4, max_bits=16)
    dummy_weights = torch.randn(100, 50)
    importance_scores = quantizer.compute_importance_score(dummy_weights)
    print(f"   Importance scores shape: {importance_scores.shape}")
    
    quantized_weights, metadata = quantizer.adaptive_quantize(dummy_weights, importance_scores)
    print(f"   Compression ratio: {metadata['compression_ratio']:.2f}x")
    print(f"   Average bits: {metadata['avg_bits']}")
    
except ImportError as e:
    print(f"ERROR: HQDE Core import failed: {e}")
    print("Make sure the HQDE project files are in the correct directory")

## 3️⃣ Test Quantum-Inspired Aggregation

In [ ]:
try:
    print("Testing QuantumInspiredAggregator...")
    
    aggregator = QuantumInspiredAggregator(noise_scale=0.01, exploration_factor=0.1)
    print("SUCCESS: QuantumInspiredAggregator created")
    
    # Create multiple weight sets (simulating different ensemble members)
    weight_list = [torch.randn(50, 30) for _ in range(4)]
    efficiency_scores = [0.9, 0.8, 0.85, 0.75]
    
    # Test aggregation
    aggregated = aggregator.efficiency_weighted_aggregation(weight_list, efficiency_scores)
    print(f"SUCCESS: Aggregation completed - shape: {aggregated.shape}")
    
    # Test quantum noise injection
    noisy_weights = aggregator.quantum_noise_injection(weight_list[0])
    noise_magnitude = torch.mean(torch.abs(noisy_weights - weight_list[0]))
    print(f"SUCCESS: Quantum noise injected - magnitude: {noise_magnitude:.6f}")
    
except Exception as e:
    print(f"ERROR: Quantum aggregator test failed: {e}")

## 4️⃣ Test Dynamic Training vs Static Behavior

In [ ]:
print("Testing Dynamic vs Static Training Behavior...")

def run_training_simulation(seed=None):
    """Run a simple training simulation and return final loss."""
    if seed is not None:
        torch.manual_seed(seed)
    
    # Simple model
    model = torch.nn.Sequential(
        torch.nn.Linear(10, 20),
        torch.nn.ReLU(),
        torch.nn.Linear(20, 1)
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    
    # Train for a few steps
    losses = []
    for step in range(10):
        # Random data
        x = torch.randn(16, 10)
        y = torch.randn(16, 1)
        
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return losses[-1], losses

# Run multiple simulations with different random seeds
results = []
for run, seed in enumerate([42, 123, 999, None], 1):
    final_loss, loss_history = run_training_simulation(seed)
    results.append(final_loss)
    print(f"Run {run} (seed={seed}): final loss = {final_loss:.6f}")

# Check if results are different (indicating dynamic behavior)
loss_variance = np.var(results)
print(f"\nVariance across runs: {loss_variance:.8f}")

if loss_variance > 1e-6:
    print("SUCCESS: DYNAMIC BEHAVIOR CONFIRMED - Results vary between runs")
else:
    print("WARNING: Results are very similar - might indicate static behavior")

# Visualize loss curves
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
for i, seed in enumerate([42, 123, 999]):
    _, loss_history = run_training_simulation(seed)
    plt.plot(loss_history, label=f'Seed {seed}', alpha=0.7)

plt.xlabel('Training Step')
plt.ylabel('Loss')
plt.title('Dynamic Training Behavior - Different Loss Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5️⃣ Test Synthetic CIFAR-10 Data Generation

In [ ]:
try:
    print("Testing Synthetic CIFAR-10 Data Generation...")
    
    # Add examples directory to path
    sys.path.insert(0, os.path.join(project_dir, 'examples'))
    from cifar10_synthetic_test import SyntheticCIFAR10DataLoader
    
    # Create synthetic data loader
    data_loader = SyntheticCIFAR10DataLoader(
        num_samples=200, 
        batch_size=32,
        num_classes=10
    )
    
    print(f"SUCCESS: Synthetic data loader created")
    print(f"   Total samples: {data_loader.num_samples}")
    print(f"   Batch size: {data_loader.batch_size}")
    print(f"   Number of batches: {len(data_loader)}")
    
    # Test data generation
    for i, (images, labels) in enumerate(data_loader):
        if i >= 3:  # Test first 3 batches
            break
        
        print(f"\n   Batch {i+1}:")
        print(f"     Images shape: {images.shape}")
        print(f"     Labels shape: {labels.shape}")
        print(f"     Image range: [{images.min():.3f}, {images.max():.3f}]")
        print(f"     Unique labels: {torch.unique(labels).tolist()}")
        
        # Display class distribution
        unique, counts = torch.unique(labels, return_counts=True)
        class_dist = dict(zip(unique.tolist(), counts.tolist()))
        print(f"     Class distribution: {class_dist}")
    
    print("\nSUCCESS: Synthetic CIFAR-10 data test completed successfully!")
    
except Exception as e:
    print(f"ERROR: Synthetic data test failed: {e}")
    print("Make sure the cifar10_synthetic_test.py file is available")

## 6️⃣ Test Real Model Predictions

In [ ]:
print("Testing Real Model Predictions vs Random Outputs...")

# Create a simple classifier
class SimpleClassifier(nn.Module):
    def __init__(self, input_size=10, num_classes=5):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create model and data
model = SimpleClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Generate test data
torch.manual_seed(42)
X_test = torch.randn(100, 10)
y_test = torch.randint(0, 5, (100,))

print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# Test before training (random predictions)
model.eval()
with torch.no_grad():
    initial_pred = model(X_test)
    _, initial_pred_classes = torch.max(initial_pred, dim=1)
    initial_accuracy = (initial_pred_classes == y_test).float().mean()

print(f"\nBefore Training:")
print(f"   Prediction accuracy: {initial_accuracy:.4f} ({initial_accuracy*100:.2f}%)")
print(f"   Prediction range: [{initial_pred.min():.3f}, {initial_pred.max():.3f}]")

# Train the model
print("\nTraining model...")
model.train()
for epoch in range(20):
    # Generate training data
    X_train = torch.randn(80, 10)
    y_train = torch.randint(0, 5, (80,))
    
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f"   Epoch {epoch+1}/20, Loss: {loss.item():.4f}")

# Test after training (learned predictions)
model.eval()
with torch.no_grad():
    final_pred = model(X_test)
    _, final_pred_classes = torch.max(final_pred, dim=1)
    final_accuracy = (final_pred_classes == y_test).float().mean()

print(f"\nAfter Training:")
print(f"   Prediction accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
print(f"   Prediction range: [{final_pred.min():.3f}, {final_pred.max():.3f}]")

# Compare results
accuracy_improvement = final_accuracy - initial_accuracy
print(f"\nResults:")
print(f"   Accuracy improvement: {accuracy_improvement:+.4f}")

if accuracy_improvement > 0.1:
    print("   SUCCESS: MODEL LEARNED - Real dynamic training confirmed!")
elif accuracy_improvement > 0:
    print("   SUCCESS: Some improvement detected - Dynamic behavior working")
else:
    print("   WARNING: No improvement - Could indicate static behavior")

# Visualize prediction distributions
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Before training
ax1.hist(initial_pred.flatten().numpy(), bins=20, alpha=0.7, color='blue')
ax1.set_title('Before Training (Random)')
ax1.set_xlabel('Prediction Values')
ax1.set_ylabel('Frequency')

# After training
ax2.hist(final_pred.flatten().numpy(), bins=20, alpha=0.7, color='green')
ax2.set_title('After Training (Learned)')
ax2.set_xlabel('Prediction Values')
ax2.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 7️⃣ Summary and Results

In [ ]:
print("HQDE SYSTEM TEST SUMMARY")
print("=" * 50)

# Count successful tests
test_results = {
    'Core Components': 'quantizer' in locals() or 'aggregator' in locals(),
    'Dynamic Training': 'loss_variance' in locals() and loss_variance > 1e-6,
    'Synthetic Data': 'data_loader' in locals(),
    'Real Predictions': 'accuracy_improvement' in locals() and accuracy_improvement > 0,
}

passed_tests = sum(test_results.values())
total_tests = len(test_results)

for test_name, result in test_results.items():
    status = "PASS" if result else "FAIL"
    print(f"   {test_name}: {status}")

print(f"\nOverall Result: {passed_tests}/{total_tests} tests passed")

if passed_tests == total_tests:
    print("\nEXCELLENT! All tests passed.")
    print("   The HQDE system is working with dynamic behavior!")
    print("\nNext steps:")
    print("   1. Install full dependencies: pip install ray psutil matplotlib")
    print("   2. Run comprehensive test: python -m hqde --mode demo")
    print("   3. Try full evaluation: python -m hqde --mode test --workers 4")
elif passed_tests >= total_tests // 2:
    print("\nGOOD PROGRESS! Most tests passed.")
    print("   The system is partially working. Check the failed tests above.")
else:
    print("\nNEEDS ATTENTION! Many tests failed.")
    print("   Please check your installation and file paths.")

print("\n" + "=" * 50)
print("HQDE Dynamic Implementation Test Complete!")